# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import models
from torchvision.models import mobilenet_v2
from tqdm import tqdm
from torchsummary import summary
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the dataset

In [ ]:
# Fix the seed to ensure reproducibility
torch.manual_seed(42)

# Data augmentation for training (applied only to the train dataset)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize to MobileNet input size
    transforms.RandomHorizontalFlip(), # Flip horizontally
    transforms.RandomRotation(15), # Random rotation 15 degress
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize((0.5,), (0.5,)) # Normalize (mean=0.5, std=0.5)
])

# No augmentation for validation/test (only resizing and normalization)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Download CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Split the test_data into test (6k) and dev (4k)
test_size = 6000
dev_size = 4000
test_dataset, dev_dataset = random_split(test_data, [test_size, dev_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Print dataset sizes
print(f"Train set size: {len(train_dataset)}")
print(f"Dev set size: {len(dev_dataset)}")
print(f"Test set size: {len(test_dataset)}")

100%|██████████| 170M/170M [00:05<00:00, 30.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Train set size: 50000
Dev set size: 4000
Test set size: 6000


# Defining the model architecture to load the pre-trained weights

In [ ]:
# Load the pre-trained MobileNet model
base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Unfreeze some of the top layers
for param in base_model.features[:-10].parameters():
    param.requires_grad = False

# Modify the classifier for CIFAR-10 (10 classes)
class MobileNetV2(nn.Module):
    def __init__(self, base_model):
        super(MobileNetV2, self).__init__()
        self.features = base_model.features
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Linear(1280, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.Linear(64, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = MobileNetV2(base_model)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 118MB/s]


In [ ]:
model_weights_path = '/content/drive/My Drive/saved_models/pytorch_models/mobilenet_cifar10.pth'
# load the full model
model.load_state_dict(torch.load(model_weights_path))
# Set the model to evaluation mode
model.eval()

print("Model weights loaded successfully!")

<ipython-input-5-ae4576b2bf67>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights_path))


Model weights loaded successfully!


## Setting device and model summary

In [ ]:
# Ensure the model is on the correct device (cuda or cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

## Required functions and classes (Evaluate and train and mask enforcing class)

In [ ]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():  # No gradients for validation (Disable gradient calculations for efficiency)
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Update loss
            test_loss += loss.item()

            # Get predictions
            _, predicted = outputs.max(1)

            # Update metrics
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    end_time = time.time()
    inference_time = end_time - start_time
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total

    #print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy, inference_time

In [ ]:
# Class to enforce pruning masks during fine-tuning
class MaskEnforcer:
    def __init__(self, model, masks):
        self.model = model
        self.masks = masks

    def enforce(self):
        with torch.no_grad():
            for name, module in self.model.named_modules():
                if name in self.masks and isinstance(module, (nn.Conv2d, nn.Linear)):
                    module.weight *= self.masks[name]

In [ ]:
# Training loop
def train_model(model, train_loader, dev_loader, criterion, optimizer, num_epochs=10, mask_enforcer=None):
    for epoch in range(num_epochs):
        ### Training Phase ###
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct, total = 0, 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Enforce pruning mask if provided
            if mask_enforcer:
                mask_enforcer.enforce()

            # Compute training metrics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Update progress bar
            loop.set_postfix(train_loss=running_loss / total, train_acc=100. * correct / total)

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total

        ### Validation (Dev) Phase ###
        model.eval()  # Set model to evaluation mode
        dev_loss, dev_correct, dev_total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in dev_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                dev_loss += loss.item()
                _, predicted = outputs.max(1)
                dev_total += labels.size(0)
                dev_correct += predicted.eq(labels).sum().item()

        dev_loss /= len(dev_loader)
        dev_acc = 100. * dev_correct / dev_total

        print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Dev Loss: {dev_loss:.4f}, Dev Acc: {dev_acc:.2f}%")

    print("Training complete!")

# Model Evaluation (before pruning)

In [ ]:
criterion = nn.CrossEntropyLoss()
avgLoss, acc, inf_time = evaluate_model(model, test_loader, criterion, device)
# Print evaluation results
print(f"Average test Loss: {avgLoss:.4f}, Test Accuracy: {acc:.2f}%, Inference Time for {len(test_dataset)} images: {inf_time:.2f} seconds")

Average test Loss: 0.2781, Test Accuracy: 91.35%, Inference Time for 6000 images: 13.13 seconds


# Pruning

## See weights before pruning

In [ ]:
# Access the first convolutional layer
conv1 = model.features[0]
print(conv1)

# Inspect its parameters (weights and bias)
print(list(conv1.named_parameters()))

Conv2dNormActivation(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU6(inplace=True)
)
[('0.weight', Parameter containing:
tensor([[[[ 1.3185e-02, -4.3213e-03,  1.4823e-02],
          [ 3.2780e-02, -2.5385e-02,  6.8572e-03],
          [ 1.0549e-02, -3.7347e-02, -1.4727e-02]],

         [[ 7.9917e-03, -5.9146e-03,  1.5076e-02],
          [ 1.9999e-02, -3.2863e-02, -2.0859e-03],
          [ 1.1350e-02, -3.2956e-02, -7.8733e-03]],

         [[-2.5234e-02, -2.0167e-02, -9.9620e-03],
          [-1.1213e-02, -2.9266e-02, -1.5218e-02],
          [-2.6531e-02, -3.3449e-02, -2.4215e-02]]],


        [[[-6.3694e-02, -2.1300e-02,  2.1416e-02],
          [ 1.3037e-01,  3.7967e-01,  4.1983e-02],
          [-1.8793e-01, -2.7921e-01, -3.8335e-02]],

         [[-6.7306e-02,  5.3145e-02,  1.8353e-03],
          [ 2.6662e-01,  7.4863e-01,  1.0166e-01],
          [-3.3641

## Apply Local unstructured pruning (magnitude pruning) (L1)

In [ ]:
import torch.nn.utils.prune as prune

# Store pruning masks
pruned_masks = {}

# Apply and store pruning masks
def prune_model(model, amount=0.2):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            prune.l1_unstructured(module, name="weight", amount=amount)
            # Save the mask for future enforcement
            pruned_masks[name] = (module.weight != 0).float()
            # Make pruning permanent
            prune.remove(module, "weight")
    return model

In [ ]:
model = prune_model(model, amount=0.5)  # Prune 30% of Conv2d and Linear layers

In [ ]:
print(pruned_masks.keys())  # Check which layers were pruned
print(pruned_masks['features.0.0'])

dict_keys(['features.0.0', 'features.1.conv.0.0', 'features.1.conv.1', 'features.2.conv.0.0', 'features.2.conv.1.0', 'features.2.conv.2', 'features.3.conv.0.0', 'features.3.conv.1.0', 'features.3.conv.2', 'features.4.conv.0.0', 'features.4.conv.1.0', 'features.4.conv.2', 'features.5.conv.0.0', 'features.5.conv.1.0', 'features.5.conv.2', 'features.6.conv.0.0', 'features.6.conv.1.0', 'features.6.conv.2', 'features.7.conv.0.0', 'features.7.conv.1.0', 'features.7.conv.2', 'features.8.conv.0.0', 'features.8.conv.1.0', 'features.8.conv.2', 'features.9.conv.0.0', 'features.9.conv.1.0', 'features.9.conv.2', 'features.10.conv.0.0', 'features.10.conv.1.0', 'features.10.conv.2', 'features.11.conv.0.0', 'features.11.conv.1.0', 'features.11.conv.2', 'features.12.conv.0.0', 'features.12.conv.1.0', 'features.12.conv.2', 'features.13.conv.0.0', 'features.13.conv.1.0', 'features.13.conv.2', 'features.14.conv.0.0', 'features.14.conv.1.0', 'features.14.conv.2', 'features.15.conv.0.0', 'features.15.conv.1

## Apply structured pruning
✅ What is `ln_structured` Doing?

- **n=2**: Uses L2 norm (Euclidean norm) for pruning. This means entire structures (e.g., channels or neurons) with the smallest L2 norm will be pruned.
- **dim=0**: Prunes across the output dimension (e.g., removes entire filters in Conv2d or neurons in Linear).

In [ ]:
import torch.nn.utils.prune as prune

# Store pruning masks
pruned_masks = {}

# Apply and store pruning masks (structured pruning)
def prune_model_structured(model, amount=0.2):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Avoid pruning critical layers
            if 'features.0' in name:
                continue

            # Apply structured pruning (removes entire channels/neurons)
            prune.ln_structured(module, name="weight", amount=amount, n=2, dim=0)

            # Store mask for later enforcement
            pruned_masks[name] = (module.weight != 0).float()

            # Make pruning permanent
            prune.remove(module, "weight")

            print(f"Pruned {name}: Remaining shape {module.weight.shape}")

    return model

In [ ]:
prune_model_structured(model, amount = 0.5)

Pruned features.1.conv.0.0: Remaining shape torch.Size([32, 1, 3, 3])
Pruned features.1.conv.1: Remaining shape torch.Size([16, 32, 1, 1])
Pruned features.2.conv.0.0: Remaining shape torch.Size([96, 16, 1, 1])
Pruned features.2.conv.1.0: Remaining shape torch.Size([96, 1, 3, 3])
Pruned features.2.conv.2: Remaining shape torch.Size([24, 96, 1, 1])
Pruned features.3.conv.0.0: Remaining shape torch.Size([144, 24, 1, 1])
Pruned features.3.conv.1.0: Remaining shape torch.Size([144, 1, 3, 3])
Pruned features.3.conv.2: Remaining shape torch.Size([24, 144, 1, 1])
Pruned features.4.conv.0.0: Remaining shape torch.Size([144, 24, 1, 1])
Pruned features.4.conv.1.0: Remaining shape torch.Size([144, 1, 3, 3])
Pruned features.4.conv.2: Remaining shape torch.Size([32, 144, 1, 1])
Pruned features.5.conv.0.0: Remaining shape torch.Size([192, 32, 1, 1])
Pruned features.5.conv.1.0: Remaining shape torch.Size([192, 1, 3, 3])
Pruned features.5.conv.2: Remaining shape torch.Size([32, 192, 1, 1])
Pruned featu

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

## See weights after pruning

In [ ]:
# Access the first convolutional layer
conv1 = model.features[0]
print(conv1)

# Inspect its parameters (weights and bias)
print(list(conv1.named_parameters()))

In [ ]:
for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and name!="model.22.dfl.conv" and name!="model.0.conv":
            print(module.weight)
            print(pruned_masks[name])

Parameter containing:
tensor([[ 0.2708,  0.0258, -0.0234,  ...,  0.0166,  0.1309, -0.0230],
        [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.1067, -0.0429, -0.0490,  ..., -0.0829,  0.3287, -0.0570],
        ...,
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [ 0.1093,  0.0201,  0.0281,  ...,  0.0527, -0.0101, -0.0232],
        [ 0.1127, -0.2928,  0.0143,  ..., -0.0595,  0.1332, -0.0287]],
       device='cuda:0', requires_grad=True)
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')
Parameter containing:
tensor([[-0.0581,  0.2195,  0.0565,  ...,  0.1554, -0.0837,  0.0481],
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0887,  0.0961,  0.0230,  ...,  0.0259,  0.1167,  0.0630

# Model Evaluation (After pruning)

In [ ]:
criterion = nn.CrossEntropyLoss()
avgLoss, acc, inf_time = evaluate_model(model, test_loader, criterion, device)
# Print evaluation results
print(f"Average test Loss: {avgLoss:.4f}, Test Accuracy: {acc:.2f}%, Inference Time for {len(test_dataset)} images: {inf_time:.2f} seconds")

Average test Loss: 2.6684, Test Accuracy: 9.92%, Inference Time for 6000 images: 11.38 seconds


# Fine Tune the pruned model

In [ ]:
# Create Mask Enforcer
mask_enforcer = MaskEnforcer(model, pruned_masks)

# Set up optimizer and criterion
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # low learning rate to prevent zero weights from getting big values

# Fine-tune the model while enforcing pruning mask
train_model(model, train_loader, dev_loader, criterion, optimizer, num_epochs=5, mask_enforcer=mask_enforcer)

Epoch 1/5: 100%|██████████| 782/782 [02:14<00:00,  5.81it/s, train_acc=27.2, train_loss=0.0329]


Epoch 1: Train Loss: 2.1012, Train Acc: 27.18%, Dev Loss: 1.6826, Dev Acc: 38.12%


Epoch 2/5: 100%|██████████| 782/782 [02:13<00:00,  5.86it/s, train_acc=36.6, train_loss=0.0277]


Epoch 2: Train Loss: 1.7738, Train Acc: 36.61%, Dev Loss: 1.5351, Dev Acc: 42.98%


Epoch 3/5: 100%|██████████| 782/782 [02:14<00:00,  5.82it/s, train_acc=40.6, train_loss=0.0258]


Epoch 3: Train Loss: 1.6518, Train Acc: 40.60%, Dev Loss: 1.4454, Dev Acc: 45.60%


Epoch 4/5: 100%|██████████| 782/782 [02:15<00:00,  5.76it/s, train_acc=43.2, train_loss=0.0245]


Epoch 4: Train Loss: 1.5666, Train Acc: 43.21%, Dev Loss: 1.3865, Dev Acc: 46.85%


Epoch 5/5: 100%|██████████| 782/782 [02:20<00:00,  5.57it/s, train_acc=44.7, train_loss=0.0236]


Epoch 5: Train Loss: 1.5067, Train Acc: 44.67%, Dev Loss: 1.3449, Dev Acc: 47.77%
Training complete!


## fine tune for more epochs

In [ ]:
# Fine-tune the model while enforcing pruning mask
train_model(model, train_loader, dev_loader, criterion, optimizer, num_epochs=5, mask_enforcer=mask_enforcer)

Epoch 1/5: 100%|██████████| 782/782 [02:16<00:00,  5.73it/s, train_acc=46.1, train_loss=0.0228]


Epoch 1: Train Loss: 1.4581, Train Acc: 46.08%, Dev Loss: 1.3118, Dev Acc: 48.62%


Epoch 2/5: 100%|██████████| 782/782 [02:15<00:00,  5.77it/s, train_acc=47, train_loss=0.0222]


Epoch 2: Train Loss: 1.4218, Train Acc: 46.99%, Dev Loss: 1.2870, Dev Acc: 49.35%


Epoch 3/5: 100%|██████████| 782/782 [02:14<00:00,  5.80it/s, train_acc=47.6, train_loss=0.0218]


Epoch 3: Train Loss: 1.3918, Train Acc: 47.62%, Dev Loss: 1.2584, Dev Acc: 50.00%


Epoch 4/5: 100%|██████████| 782/782 [02:13<00:00,  5.84it/s, train_acc=48.2, train_loss=0.0213]


Epoch 4: Train Loss: 1.3618, Train Acc: 48.20%, Dev Loss: 1.2409, Dev Acc: 50.33%


Epoch 5/5: 100%|██████████| 782/782 [02:15<00:00,  5.78it/s, train_acc=48.5, train_loss=0.021]


Epoch 5: Train Loss: 1.3421, Train Acc: 48.45%, Dev Loss: 1.2263, Dev Acc: 50.73%
Training complete!


## Check if pruned weights still zero after fine-tuning

In [ ]:
for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and name!="model.22.dfl.conv" and name!="model.0.conv":
            print(module.weight)
            print(pruned_masks[name])

Parameter containing:
tensor([[ 0.2708,  0.0093,  0.0333,  ...,  0.0166,  0.1392, -0.0230],
        [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.1067, -0.0475, -0.0326,  ..., -0.0829,  0.3345, -0.0570],
        ...,
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [ 0.1093, -0.0117, -0.0108,  ...,  0.0527, -0.0345, -0.0232],
        [ 0.1127, -0.2593,  0.0379,  ..., -0.0595,  0.1224, -0.0287]],
       device='cuda:0', requires_grad=True)
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')
Parameter containing:
tensor([[-0.0421,  0.2191,  0.0641,  ...,  0.1631, -0.0823,  0.0510],
        [ 0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.0868,  0.0919,  0.0260,  ...,  0.0540,  0.1047,  0.0538

In [ ]:
# Verify pruning levels
def check_sparsity(model):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            total = module.weight.numel()
            nonzero = module.weight.nonzero().size(0)
            sparsity = 100 * (1 - nonzero / total)
            print(f"{name}: {sparsity:.2f}% sparsity")

check_sparsity(model)

features.0.0: 0.00% sparsity
features.1.conv.0.0: 50.00% sparsity
features.1.conv.1: 50.00% sparsity
features.2.conv.0.0: 50.00% sparsity
features.2.conv.1.0: 50.00% sparsity
features.2.conv.2: 50.00% sparsity
features.3.conv.0.0: 50.00% sparsity
features.3.conv.1.0: 50.00% sparsity
features.3.conv.2: 50.00% sparsity
features.4.conv.0.0: 50.00% sparsity
features.4.conv.1.0: 50.00% sparsity
features.4.conv.2: 50.00% sparsity
features.5.conv.0.0: 50.00% sparsity
features.5.conv.1.0: 50.00% sparsity
features.5.conv.2: 50.00% sparsity
features.6.conv.0.0: 50.00% sparsity
features.6.conv.1.0: 50.00% sparsity
features.6.conv.2: 50.00% sparsity
features.7.conv.0.0: 50.00% sparsity
features.7.conv.1.0: 50.00% sparsity
features.7.conv.2: 50.00% sparsity
features.8.conv.0.0: 50.00% sparsity
features.8.conv.1.0: 50.00% sparsity
features.8.conv.2: 50.00% sparsity
features.9.conv.0.0: 50.00% sparsity
features.9.conv.1.0: 50.00% sparsity
features.9.conv.2: 50.00% sparsity
features.10.conv.0.0: 50.00

# Evaluate model after fine tuning

In [ ]:
criterion = nn.CrossEntropyLoss()
avgLoss, acc, inf_time = evaluate_model(model, test_loader, criterion, device)
# Print evaluation results
print(f"Average test Loss: {avgLoss:.4f}, Test Accuracy: {acc:.2f}%, Inference Time for {len(test_dataset)} images: {inf_time:.2f} seconds")

Average test Loss: 1.2260, Test Accuracy: 51.05%, Inference Time for 6000 images: 13.09 seconds


# Convert pruned layers to sparse

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.Conv2d):
        module.weight = torch.nn.Parameter(module.weight.to_sparse())  # Convert to sparse format

In [ ]:
for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and name!="model.22.dfl.conv" and name!="model.0.conv":
            print(module.weight)
            print(pruned_masks[name])

Parameter containing:
tensor(indices=tensor([[   0,    0,    0,  ...,  255,  255,  255],
                       [   0,    1,    2,  ..., 1277, 1278, 1279]]),
       values=tensor([ 0.2708,  0.0093,  0.0333,  ..., -0.0595,  0.1224,
                      -0.0287]),
       device='cuda:0', size=(256, 1280), nnz=163840, layout=torch.sparse_coo,
       requires_grad=True)
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')
Parameter containing:
tensor(indices=tensor([[  0,   0,   0,  ..., 127, 127, 127],
                       [  0,   1,   2,  ..., 253, 254, 255]]),
       values=tensor([-0.0421,  0.2191,  0.0641,  ...,  0.1467,  0.0399,
                      -0.0022]),
       device='cuda:0', size=(128, 256), nnz=16384, layout=torch.sparse_coo,
       requires_grad=True)
ten

In [ ]:
model_save_path = '/content/drive/My Drive/saved_models/pytorch_models/mobilenet_sparse.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model weights saved successfully at: {model_save_path}")

Model weights saved successfully at: /content/drive/My Drive/saved_models/pytorch_models/mobilenet_sparse.pth
